In [1]:
import json
import re

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from textblob import TextBlob

In [2]:
with open('dataset/goodreads_reviews_spoiler.json') as json_file:
    data = json.loads('[' + ',\n'.join(json_file.readlines()) + ']')

# Preprocess

In [3]:
def unpack(sents):
    ys = []
    xs = []
    for y, sent in sents:
        ys.append(y)
        xs.append(sent)
    return ys, xs

In [4]:
x = []
y = []
for sample in tqdm(data):
    ans, sent = unpack(sample['review_sentences'])
    x.extend(sent)
    y.extend(ans)

100%|██████████| 1378033/1378033 [00:04<00:00, 342278.32it/s]


In [5]:
del data

In [6]:
for id, i in tqdm(enumerate(x), total=len(x)):
    text = re.sub(r'http\S+', '', i.lower())
    # text = str(TextBlob(text).correct())
    x[id] = text

100%|██████████| 17672655/17672655 [00:34<00:00, 513027.07it/s]


In [7]:
prep_data = pd.DataFrame([x,y], index=['text', 'target']).T

In [13]:
prep_data = prep_data[~(prep_data['text'] == '')]

In [15]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(prep_data, test_size=0.2)

In [16]:
train.to_csv('dataset/baseline/train.csv', index=False)
test.to_csv('dataset/baseline/test.csv', index=False)

# TF-IDF baseline

In [1]:
import text_clf

In [2]:
CONFIG = 'config/baseline_config.yaml'

In [3]:
model, target_names_mapping = text_clf.train(path_to_config=CONFIG)

text-clf-train - INFO - Config:

seed: 42
path_to_save_folder: models
experiment_name: model

# data
data:
  train_data_path: dataset/baseline/train.csv
  test_data_path: dataset/baseline/test.csv
  sep: ','
  text_column: text
  target_column: target

# preprocessing
# (included in resulting model pipeline, so preserved for inference)
preprocessing:
  lemmatization: null  # pymorphy2

# tf-idf
tf-idf:
  lowercase: true
  ngram_range: (1, 1)
  max_df: 1.0
  min_df: 1

# logreg
logreg:
  penalty: l2
  C: 1.0
  class_weight: balanced
  solver: saga
  n_jobs: -1

# grid-search
grid-search:
  do_grid_search: false
  grid_search_params_path: hyperparams.py
text-clf-train - INFO - Loading data...
text-clf-train - INFO - Train dataset size: 14127218
text-clf-train - INFO - Test dataset size: 3531805
text-clf-train - INFO - Fitting TF-IDF + LogReg model...
Traceback (most recent call last):
  File "/home/xx/anaconda3/envs/diploma/lib/python3.8/site-packages/text_clf/__main__.py", line 33, in t